In [1]:
from preprocessing.settings import *
from keras.models import load_model
import numpy as np
from numpy import array
import _pickle as pickle
import os

# import data_processing
from preprocessing import chord_model
from preprocessing import midi_functions as mf
from preprocessing import data_class

Using TensorFlow backend.


In [39]:
chord_model_folder = 'models/chords/'   
chord_model_name = 'model_Epoch250_4.pickle'

melody_model_folder = 'models/chords_mldy/'
melody_model_name = 'modelEpoch500.pickle'

midi_save_folder = 'predicted_midi/'

#'data/' + shift_folder + 'indroll/'
seed_path = "G:/MSc/sem-4/project/temp/JamBot-master/OWN/data/shifted/indroll/2Boys/I Won't Let You Down.mid.pickle"
#'data/' + shift_folder + 'chord_index/'
seed_chord_path = "G:/MSc/sem-4/project/temp/JamBot-master/OWN/data/shifted/chord_index/2Boys/I Won't Let You Down.mid.pickle"

seed_name = "I Won't Let You Down.mid.pickle"

In [40]:
# Parameters for song generation:
BPM = 90
note_cap = 5
chord_temperature = 1

# Params for seed:
# length of the predicted song in bars:
num_bars =64
# The first seed_length number of bars from the seed will be used: 
seed_length = 4

#pred_song_length = 8*16-seed_length

In [41]:
# data_processing
def get_chord_dict():
    chord_to_index = pickle.load(open(dict_path + chord_dict_name, 'rb'))
    index_to_chord = pickle.load(open(dict_path + index_dict_name, 'rb'))
    return chord_to_index, index_to_chord

In [42]:
with_seed = True

chord_to_index, index_to_chord = get_chord_dict()

In [43]:
# chord_to_index, index_to_chord

In [44]:
def sample_probability_vector(prob_vector):
    # Sample a probability vector, e.g. [0.1, 0.001, 0.5, 0.9]
            
    sum_probas = sum(prob_vector)
    
    
    if sum_probas > note_cap:
        prob_vector = (prob_vector/sum_probas)*note_cap
    
    note_vector = np.zeros((prob_vector.size), dtype=np.int8)
    for i, prob in enumerate(prob_vector):
        note_vector[i] = np.random.multinomial(1, [1 - prob, prob])[1]
    return note_vector

In [45]:
def ind_to_onehot(ind):
    onehot = np.zeros((len(ind), num_notes))
    for i, step in enumerate(ind):
        for note in step:
            onehot[i,note]=1
    return onehot
print(seed_name)
sd = pickle.load(open(seed_path, 'rb'))[:8*seed_length]
seed_chords = pickle.load(open(seed_chord_path, 'rb'))[:seed_length]

seed = ind_to_onehot(sd)[:,low_crop:high_crop]

I Won't Let You Down.mid.pickle


In [46]:
print('loading polyphonic model ...')
melody_model = load_model(melody_model_folder+melody_model_name)
melody_model.reset_states()

ch_model = chord_model.Chord_Model(
        chord_model_folder+chord_model_name,
        prediction_mode='sampling',
        first_chords=seed_chords,
        temperature=chord_temperature)

loading polyphonic model ...
loading chord model ...


C:\Users\Dax\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [47]:
chords = []

for i in range((num_bars+2)):
    ch_model.predict_next()


if chord_embed_method == 'embed':
    embedded_chords = ch_model.embed_chords_song(ch_model.song)
elif chord_embed_method == 'onehot':
    embedded_chords = data_class.make_one_hot_vector(ch_model.song, num_chords)
elif chord_embed_method == 'int':
    embedded_chords = [[x] for x in ch_model.song]

[-0.02178893 -0.02834796  0.01632705  0.06109789 -0.04774819  0.06711037
  0.00886357  0.01567437  0.04337154  0.04299917]
[-0.02178893 -0.02834796  0.01632705  0.06109789 -0.04774819  0.06711037
  0.00886357  0.01567437  0.04337154  0.04299917]
[-0.02178893 -0.02834796  0.01632705  0.06109789 -0.04774819  0.06711037
  0.00886357  0.01567437  0.04337154  0.04299917]
[ 0.08988088  0.07731621 -0.12642536 -0.1529983   0.08236459  0.02992695
 -0.01494367 -0.03369865 -0.18186645  0.15129308]
[-0.07797422 -0.04547906  0.10006183  0.01637116 -0.04989265  0.00297065
  0.00667679  0.03888008  0.12503724 -0.02752901]
[-0.04938662  0.03335866  0.06054105 -0.00894505  0.01407535  0.09018091
  0.03531275  0.05795756  0.07534554  0.0715729 ]
[-0.07797422 -0.04547906  0.10006183  0.01637116 -0.04989265  0.00297065
  0.00667679  0.03888008  0.12503724 -0.02752901]
[ 0.16982038  0.08945112  0.12174977  0.16274048  0.0904616  -0.11959809
 -0.20210303 -0.17346382  0.11172931  0.00828196]
[ 0.02351    -0.

In [48]:
chords = []

for j in range((len(ch_model.song)-2)*fs*2):
    ind = int(((j+1)/(fs*2)))
    if next_chord_feature:
        ind2 = int(((j+1)/(fs*2)))+1
#        print(j)
#        print(ind, ' ', ind2)
        chords.append(list(embedded_chords[ind])+list(embedded_chords[ind2]))
    else:
        chords.append(embedded_chords[ind])

#    print(ind)

In [49]:
chords=np.array(chords)

if counter_feature:
    counter = [[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]]
    counter = np.array(counter*(len(ch_model.song)-2))
    chords = np.append(chords, counter, axis=1)


seed = np.append(seed, chords[:seed.shape[0]], axis=1)    

seed = np.reshape(seed, (seed.shape[0], 1, 1, seed.shape[1]))

next_step = None

for step in seed:
    
    next_step = melody_model.predict(step)
    
    
notes = sample_probability_vector(next_step[0])

rest = []
rest.append(notes)


for chord in chords[seed.shape[0]:]:
    next_input = np.append(notes, chord, axis=0)
    next_input = np.reshape(next_input, (1, 1, next_input.shape[0]))
    next_step = melody_model.predict(next_input)
    notes = sample_probability_vector(next_step[0])
    rest.append(notes)

rest = np.array(rest)
rest = np.pad(rest, ((0,0),(low_crop,num_notes-high_crop)), mode='constant', constant_values=0)
ind = np.nonzero(rest)
#rest = np.reshape(rest, (rest.shape[1], rest.shape[0]))
#note_ind = mf.pianoroll_to_note_index(rest)
#print(ch_model.song)

In [32]:
instrument_names = ['Electric Guitar (jazz)', 'Acoustic Grand Piano',
'Bright Acoustic Piano', 'Electric Piano 1',
'Honky-tonk Piano', 'Glockenspiel',
'Acoustic Guitar (nylon)', 'Acoustic Guitar (steel)',
'Orchestral Harp', 'String Ensemble 2', 'SynthStrings 1', 'SynthStrings 2']

In [33]:
for instrument_name in instrument_names:
    mf.pianoroll_to_midi_continous(rest, midi_save_folder, instrument_name, instrument_name, BPM)

In [ ]:
# 50 - 100

In [17]:
rest.shape

(513, 128)

In [21]:
ind[0].shape, ind[1].shape

((1417,), (1417,))

In [23]:
rest_index = []
for i in rest:
    for j in i:
        if j != 0:
            rest_index.append(j)
            

In [25]:
len(rest_index)

1417

In [ ]:
# 500 - 1000

In [37]:
# NEW

rest.shape

(513, 128)

In [38]:
ind[0].shape, ind[1].shape

((224,), (224,))

In [50]:
# 250 - 500

In [51]:
rest.shape

(513, 128)

In [52]:
ind[0].shape, ind[1].shape

((404,), (404,))